# Preprocessing
---

In [152]:
import pandas as pd
import numpy as np

In [140]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    return x.rjust(6, "0")

# 2. 중복인 거래소코드(column) 찾기
def find_overlap(df, column):
    NCAV1_name = df[column].unique().tolist()
    rm_list = list()

    for i in NCAV1_name:
        if (len(df[df[column]==i]) > 1):
            rm_list.append(i)
    return rm_list

# 1. 펀더멘탈지표
- NCAV : (유동자산 - 총부채) / 시가총액
    - 벤자민 그레이엄의 핵심 지표
- PEGR : **PER(전일_21년도3월31일)** / **EPS 증가율(3년평균)**
    - 피터 린치의 핵심 지표
- PSR : **현재주가(21.03.31)** / **1주당 매출액(20년)**
    - 켄 피셔의 핵심 지표
- **유동비율(20년도)**
    - 유동자산의 유동부채에 대한 비율
- **부채비율(20년도)**

## 1-1. NCAV
> (유동자산 - 총부채) / 시가총액

### 1-1-1. 유동자산_총부채 전처리

In [211]:
NCAV1 = pd.read_csv("./datasets/raw/NCAV(유동자산_총부채).csv", encoding="CP949")
NCAV1

,회사명,거래소코드,회계년도,유동자산(요약)(백만원),부채총계(요약)(백만원)
0,(주)BNK금융지주,138930,2020/12,NaN,NaN
1,(주)CMG제약,58820,2020/12,145981.0,14225.0
2,(주)DB하이텍,990,2020/12,455643.0,384020.0
3,(주)ES큐브,50120,2020/12,7524.0,9046.0
4,(주)HJ중공업,97230,2020/12,1067320.0,1868707.0
...,...,...,...,...,...
2303,휴마시스(주),205470,2020/12,42484.0,19508.0
2304,휴젤(주),145020,2020/12,552590.0,191023.0
2305,흥구석유(주),24060,2020/12,27688.0,4121.0
2306,흥국화재해상보험(주),540,2020/12,NaN,NaN


In [212]:
# 중복인 거래소코드 찾기
find_overlap(NCAV1, "거래소코드")

[108320]

In [213]:
NCAV1[NCAV1["거래소코드"]==108320]

,회사명,거래소코드,회계년도,유동자산(요약)(백만원),부채총계(요약)(백만원)
744,(주)엘엑스세미콘,108320,2020/12,627617.0,218870.0
745,(주)엘엑스세미콘,108320,2020/12,627617.0,218870.0


In [214]:
NCAV1.drop(index=744, axis=0, inplace=True) # 중복회사 데이터 단일화 완료
NCAV1.columns = ['회사명', '종목코드', '회계년도', '유동자산(백만원)', '부채총계(백만원)']
NCAV1["종목코드"] = NCAV1["종목코드"].astype("str")
NCAV1["종목코드"] = NCAV1["종목코드"].map(padding)
NCAV1

,회사명,종목코드,회계년도,유동자산(백만원),부채총계(백만원)
0,(주)BNK금융지주,138930,2020/12,NaN,NaN
1,(주)CMG제약,058820,2020/12,145981.0,14225.0
2,(주)DB하이텍,000990,2020/12,455643.0,384020.0
3,(주)ES큐브,050120,2020/12,7524.0,9046.0
4,(주)HJ중공업,097230,2020/12,1067320.0,1868707.0
...,...,...,...,...,...
2303,휴마시스(주),205470,2020/12,42484.0,19508.0
2304,휴젤(주),145020,2020/12,552590.0,191023.0
2305,흥구석유(주),024060,2020/12,27688.0,4121.0
2306,흥국화재해상보험(주),000540,2020/12,NaN,NaN


### 1-1-2. 시가총액 전처리

In [215]:
NCAV2_kospi = pd.read_csv("./datasets/raw/KOSPI_시가총액(20210331).csv", encoding="CP949")
NCAV2_kosdaq = pd.read_csv("./datasets/raw/KOSDAQ_시가총액(20210331).csv", encoding="CP949")
NCAV2 = pd.concat([NCAV2_kospi,NCAV2_kosdaq], axis=0).reset_index()
NCAV2 = NCAV2[["종목코드", "시가총액"]]
NCAV2["종목코드"] = NCAV2["종목코드"].map(padding)
NCAV2["시가총액"]= NCAV2["시가총액"]/1000000
NCAV2

,종목코드,시가총액
0,095570,2.109340e+05
1,006840,3.537100e+05
2,027410,6.709750e+05
3,282330,2.730860e+06
4,138930,2.213100e+06
...,...,...
2414,024060,1.087500e+05
2415,010240,1.168190e+05
2416,189980,2.080580e+05
2417,037440,6.620137e+04


### 1-1-3. NCAV 계산

In [216]:
df_NCAV = pd.merge(NCAV1, NCAV2, on=['종목코드'], how = 'left') # 회사수가 적은 쪽을 기준으로 합친다.(2307 vs 2419)
df_NCAV

,회사명,종목코드,회계년도,유동자산(백만원),부채총계(백만원),시가총액
0,(주)BNK금융지주,138930,2020/12,NaN,NaN,2.213100e+06
1,(주)CMG제약,058820,2020/12,145981.0,14225.0,6.090420e+05
2,(주)DB하이텍,000990,2020/12,455643.0,384020.0,2.499640e+06
3,(주)ES큐브,050120,2020/12,7524.0,9046.0,6.944812e+04
4,(주)HJ중공업,097230,2020/12,1067320.0,1868707.0,6.212260e+05
...,...,...,...,...,...,...
2302,휴마시스(주),205470,2020/12,42484.0,19508.0,2.313530e+05
2303,휴젤(주),145020,2020/12,552590.0,191023.0,2.228650e+06
2304,흥구석유(주),024060,2020/12,27688.0,4121.0,1.087500e+05
2305,흥국화재해상보험(주),000540,2020/12,NaN,NaN,2.637160e+05


In [217]:
# (유동자산 - 총부채) / 시가총액
df_NCAV["NCAV"] = (df_NCAV["유동자산(백만원)"] - df_NCAV["부채총계(백만원)"]) / df_NCAV["시가총액"] # 컬럼생성
df_NCAV.drop(['유동자산(백만원)', '부채총계(백만원)', '시가총액'], axis=1, inplace=True) # 기존 컬럼 제거
df_NCAV.dropna(axis=0, inplace=True) # 결측치 제거
df_NCAV # 끝!

,회사명,종목코드,회계년도,NCAV
1,(주)CMG제약,058820,2020/12,0.216333
2,(주)DB하이텍,000990,2020/12,0.028653
3,(주)ES큐브,050120,2020/12,-0.021916
4,(주)HJ중공업,097230,2020/12,-1.290009
7,(주)MH에탄올,023150,2020/12,-0.301897
...,...,...,...,...
2301,휴림로봇(주),090710,2020/12,-0.133651
2302,휴마시스(주),205470,2020/12,0.099311
2303,휴젤(주),145020,2020/12,0.162236
2304,흥구석유(주),024060,2020/12,0.216708


## 1-2. PEGR
> PER(전일_21년도3월31일) / EPS 증가율(3년평균)

### 1-2-1. PER 전처리

In [134]:
PEGR1 = pd.read_csv("./datasets/raw/KOSPI_PER_PBR_20210331.csv", index_col=0)
PEGR1["종목코드"] = PEGR1["종목코드"].map(padding)
PEGR1 = PEGR1[["종목코드", "PER"]]

PEGR2 = pd.read_csv("./datasets/raw/KOSDAQ_PER_PBR_20210331.csv", index_col=0)
PEGR2["종목코드"] = PEGR2["종목코드"].map(padding)
PEGR2 = PEGR2[["종목코드", "PER"]]

PEGR_PER = pd.concat([PEGR1,PEGR2], axis=0)
PEGR_PER

,종목코드,PER
0,095570,4.59
1,006840,12.32
2,027410,24.95
3,282330,18.03
4,138930,4.12
...,...,...
1469,024060,12.72
1470,010240,16.69
1471,189980,35.89
1472,037440,16.51


### 1-2-2. EPS 전처리

> # 제조라고 적혀있는데 괜찮나?

> # 3년 증가율 어떻게 계산? 20년-18년 EPS이렇게 구하나? 19년 필요없나?

In [136]:
PEGR3 = pd.read_csv("./datasets/raw/EPS 증가율(2018_2020).csv", encoding="CP949")
PEGR3

,회사명,거래소코드,회계년도,[제조]EPS(Earning Per Share)
0,(주)BNK금융지주,138930,2018/12,NaN
1,(주)BNK금융지주,138930,2019/12,NaN
2,(주)BNK금융지주,138930,2020/12,NaN
3,(주)CMG제약,58820,2018/12,56.0
4,(주)CMG제약,58820,2019/12,25.0
...,...,...,...,...
6849,흥국화재해상보험(주),540,2019/12,NaN
6850,흥국화재해상보험(주),540,2020/12,NaN
6851,흥아해운(주),3280,2018/12,-576.0
6852,흥아해운(주),3280,2019/12,-400.0


In [146]:
PEGR3.dropna(axis=0, inplace=True)
PEGR3

,회사명,거래소코드,회계년도,[제조]EPS(Earning Per Share)
3,(주)CMG제약,58820,2018/12,56.0
4,(주)CMG제약,58820,2019/12,25.0
5,(주)CMG제약,58820,2020/12,2.0
6,(주)DB하이텍,990,2018/12,2192.0
7,(주)DB하이텍,990,2019/12,2790.0
...,...,...,...,...
6846,흥구석유(주),24060,2019/12,570.0
6847,흥구석유(주),24060,2020/12,330.0
6851,흥아해운(주),3280,2018/12,-576.0
6852,흥아해운(주),3280,2019/12,-400.0


In [150]:
EPS_list = PEGR3["회계년도"].unique().to_list()

array(['2018/12', '2019/12', '2020/12', '2018/09', '2019/09', '2020/09',
       '2018/06', '2019/06', '2020/06', '2018/03', '2019/03', '2020/03',
       '2018/02', '2019/02', '2020/02', '2018/08', '2019/08', '2020/08',
       '2018/11', '2019/11', '2020/11'], dtype=object)

In [149]:
PEGR3.groupby("회사명").mean()

,거래소코드,[제조]EPS(Earning Per Share)
회사명,,
(주)CMG제약,58820.0,27.666667
(주)DB하이텍,990.0,2932.000000
(주)ES큐브,50120.0,-989.666667
(주)HJ중공업,97230.0,-3139.000000
(주)MH에탄올,23150.0,127.000000
...,...,...
휴림로봇(주),90710.0,26.666667
휴마시스(주),205470.0,190.666667
휴젤(주),145020.0,8032.000000


## 1-3. PSR
> 현재주가(21.03.31) / 1주당 매출액(20년)

### 1-3-1. 현재주가 전처리

In [181]:
PSR1 = pd.read_csv("./datasets/raw/현재주가(210331).csv", encoding="CP949")
PSR1.columns = ['회사명', '종목코드', '회계년도', '종가']
PSR1["종목코드"] = PSR1["종목코드"].astype("str")
PSR1["종목코드"] = PSR1["종목코드"].map(padding)
PSR1 = PSR1[['회사명', '종목코드', '종가']]
PSR1

,회사명,종목코드,종가
0,AJ네트웍스보통주,095570,4505
1,AK홀딩스보통주,006840,26700
2,BGF리테일보통주,282330,158000
3,BGF보통주,027410,7010
4,BNK금융지주보통주,138930,6790
...,...,...,...
1743,웰바이오텍보통주,010600,2825
1744,웰크론,065950,4980
1745,인지컨트롤스보통주,023800,14500
1746,인팩보통주,023810,6890


### 1-3-2. 1주당 매출액 전처리

In [220]:
PSR2 = pd.read_csv("./datasets/raw/1주당 매출액(2020).csv", encoding="CP949")
PSR2.columns = ['회사명', '종목코드', '회계년도', '[제조]1주당매출액']
PSR2["종목코드"] = PSR2["종목코드"].astype("str")
PSR2["종목코드"] = PSR2["종목코드"].map(padding)
PSR2 = PSR2[['종목코드', '[제조]1주당매출액']]
# PSR2["회계년도"].value_counts()
PSR2

,종목코드,[제조]1주당매출액
0,138930,NaN
1,058820,464.10
2,000990,21026.63
3,050120,363.68
4,097230,20350.03
...,...,...
2327,205470,1335.83
2328,145020,42537.34
2329,024060,7656.25
2330,000540,NaN


In [196]:
find_overlap(PSR2, "종목코드")

['108320']

In [197]:
PSR2[PSR2["종목코드"]=="108320"]

,종목코드,[제조]1주당매출액
752,108320,71438.44
753,108320,71438.44


In [198]:
PSR2.drop(index=752, inplace=True)
PSR2

,종목코드,[제조]1주당매출액
0,138930,NaN
1,058820,464.10
2,000990,21026.63
3,050120,363.68
4,097230,20350.03
...,...,...
2327,205470,1335.83
2328,145020,42537.34
2329,024060,7656.25
2330,000540,NaN


### 1-3-3. PSR 계산

> # 금융, 보험, 지주 제외하고 결측치 다시 확인

In [201]:
df_PSR = pd.merge(PSR1, PSR2, on=['종목코드'], how = 'left') # 회사수가 적은 쪽을 기준으로 합친다.(1748 vs 2331)
# df_PSR.dropna(inplace=True)
df_PSR

,회사명,종목코드,종가,[제조]1주당매출액
0,AJ네트웍스보통주,095570,4505,9686.73
1,AK홀딩스보통주,006840,26700,3435.73
2,BGF리테일보통주,282330,158000,356852.72
3,BGF보통주,027410,7010,539.44
4,BNK금융지주보통주,138930,6790,NaN
...,...,...,...,...
1743,웰바이오텍보통주,010600,2825,377.43
1744,웰크론,065950,4980,4380.94
1745,인지컨트롤스보통주,023800,14500,20556.88
1746,인팩보통주,023810,6890,17782.77


In [202]:
df_PSR.isna().sum()

회사명             0
종목코드            0
종가              0
[제조]1주당매출액    474
dtype: int64

## 1-4. 유동비율 & 부채비율

In [204]:
유부비율 = pd.read_csv("./datasets/raw/부채비율_유동비율(2020).csv", encoding="CP949")

유부비율["종목코드"] = 유부비율["종목코드"].map(padding)


,회사명,거래소코드,회계년도,부채비율,유동비율
0,(주)BNK금융지주,138930,2020/12,NaN,NaN
1,(주)CMG제약,58820,2020/12,7.85,1069.54
2,(주)DB하이텍,990,2020/12,48.29,174.09
3,(주)ES큐브,50120,2020/12,12.40,90.15
4,(주)HJ중공업,97230,2020/12,583.21,103.45
...,...,...,...,...,...
2327,휴마시스(주),205470,2020/12,39.94,439.90
2328,휴젤(주),145020,2020/12,28.38,879.33
2329,흥구석유(주),24060,2020/12,5.18,1108.08
2330,흥국화재해상보험(주),540,2020/12,NaN,NaN


## 1-5. 펀더멜탈 지표 merge